# Day 5

In [1]:
# Read input file
with open('../inputs/adventofcode.com_2023_day_05_input.txt', 'r') as f:
    data = f.read().splitlines()

## Puzzle 1

In [2]:
import re
maps = {}
maps_names = ['seed-to-soil', 'soil-to-fertilizer', 'fertilizer-to-water', 'water-to-light', 'light-to-temperature', 'temperature-to-humidity', 'humidity-to-location']
for n in maps_names:
    maps[n] = []


for line in data:
    if line.startswith('seeds:'):
        seeds = list(map(int, re.findall(r'\d+', line)))
    elif line.startswith('seed-to-soil map:'):
        mode = 'seed-to-soil'
    elif line.startswith('soil-to-fertilizer map:'):
        mode = 'soil-to-fertilizer'
    elif line.startswith('fertilizer-to-water map:'):
        mode = 'fertilizer-to-water'
    elif line.startswith('water-to-light map:'):
        mode = 'water-to-light'
    elif line.startswith('light-to-temperature map:'):
        mode = 'light-to-temperature'
    elif line.startswith('temperature-to-humidity map:'):
        mode = 'temperature-to-humidity'
    elif line.startswith('humidity-to-location map:'):
        mode = 'humidity-to-location'
    else:
        if line:
            dest_start, source_start, length = map(int, re.findall(r'\d+', line))
        else:
            continue

        maps[mode].append((source_start, dest_start, length))
        
locations = []
# For each seed, follow the maps in order
for s in seeds:
    lookup = s
    for n in maps_names:
        for source_start, dest_start, length in maps[n]:
            if lookup in range(source_start, source_start+length): # Found if the lookup is in any range
                lookup = dest_start - source_start + lookup
                break
    locations.append((lookup))

print(f'The lowest location number for a seed is {min(locations)}.')

The lowest location number for a seed is 346433842.


## Puzzle 2

In [3]:
def get_partial_ranges(start_range, end_range, map_name):   # 79, 93
    ranges = []
    for source_start, dest_start, length in maps[map_name]:

        # Partial overlap from beginning
        if start_range in range(source_start, source_start+length):
            new_source_start = start_range
            new_dest_start = dest_start - source_start + new_source_start
            # Lookup range end is within the source range
            if source_start + length < end_range:
                l = source_start + length - start_range
            # All the source range is within the lookup range
            else:
                l = end_range - start_range
            ranges.append((new_source_start, new_dest_start, l))

        elif source_start in range(start_range, end_range):
            new_source_start = source_start
            new_dest_start = dest_start
            # Lookup range end is fully within the source range
            if source_start + length < end_range:
                l = length
            # Lookup range end is partially within the source range
            else:
                l = end_range - source_start
            ranges.append((new_source_start, new_dest_start, l))

        elif source_start > end_range:
            break

    # Ensure ranges are sorted
    ranges.sort(key=lambda x: x[0])

    # Fill 1:1 mapping (gaps)
    a = start_range
    for r in ranges:
        d = r[0] - a
        if d > 0:
            ranges.append((a, a, d))
        a = r[0] + r[2]
    if a < end_range:
        ranges.append((a, a, end_range - a))

    ranges.sort(key=lambda x: x[0])
    return ranges


# Start by making sure the ranges in the maps are sorted
for map_name in maps_names:
    maps[map_name].sort(key=lambda x: x[0])

# Ranges of interest
seed_ranges = [(seeds[i], seeds[i] + seeds[i+1]) for i in range(0, len(seeds), 2)]

# The ranges of interest are the seed ranges at first and then the corresponding soil, fertilizer...
ranges_of_interest = []
for start_range, end_range in seed_ranges:
    ranges_of_interest.extend(get_partial_ranges(start_range, end_range, 'seed-to-soil'))

# For each map, get the corresponding partial ranges
for map_name in maps_names[1:]:
    new_ranges_of_interest = []
    for src, dst, l in ranges_of_interest:
        p_ranges = get_partial_ranges(dst, dst+l, map_name)
        new_ranges_of_interest.extend(p_ranges)
    ranges_of_interest = new_ranges_of_interest

ranges_of_interest.sort(key=lambda x: x[1])
print(f'The minimum location of the seeds is {ranges_of_interest[0][1]}.')


The minimum location of the seeds is 60294664.
